In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from google.colab import files

In [2]:
# importing data
df = pd.read_csv('output.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   URL Links            141 non-null    object
 1   Department Initials  141 non-null    object
 2   Title                1 non-null      object
 3   Catalog Page         1 non-null      object
dtypes: object(4)
memory usage: 4.5+ KB


,URL Links,Department Initials,Title,Catalog Page
0,https://www.utdallas.edu/fact-sheets/is/ba-ame...,is,Bachelor of Arts in American Studies,https://catalog.utdallas.edu/now/undergraduate...
1,https://www.utdallas.edu/fact-sheets/is/bs-hea...,is,NaN,NaN
2,https://www.utdallas.edu/fact-sheets/is/ba-bs-...,is,NaN,NaN
3,https://www.utdallas.edu/fact-sheets/atec/ba-a...,atec,NaN,NaN
4,https://www.utdallas.edu/fact-sheets/bbs/bs-co...,bbs,NaN,NaN


Note that the robots.txt of utdallas.edu asks for a crawl-delay of 30s; also, when opening any of the URL links, note that:
<ol>
  <li>The title is the only thing with the <strong>h1 tag</strong></li>
  <li>The catalog page link is an href that is <strong>an anchor tag with id="catalog"</strong></li>
</ol>

In [3]:
# creating new columns
df['Title'] = ['' for _ in range(df.shape[0])]
df['Catalog Page'] = ['' for _ in range(df.shape[0])]
df.head()

,URL Links,Department Initials,Title,Catalog Page
0,https://www.utdallas.edu/fact-sheets/is/ba-ame...,is,,
1,https://www.utdallas.edu/fact-sheets/is/bs-hea...,is,,
2,https://www.utdallas.edu/fact-sheets/is/ba-bs-...,is,,
3,https://www.utdallas.edu/fact-sheets/atec/ba-a...,atec,,
4,https://www.utdallas.edu/fact-sheets/bbs/bs-co...,bbs,,


In [4]:
# now scraping all links, should take around 140x30 seconds = 4200 seconds = 70 minutes
for i, link in enumerate(df['URL Links']):
  page = requests.get(link)
  soup = BeautifulSoup(page.content, 'html.parser')
  df['Catalog Page'][i] = soup.find(id='catalog', href=True)['href']
  df['Title'][i] = soup.find('h1').text
  df.to_csv('new output.csv',index=False)
  print(df.iloc[i])
  time.sleep(30)

URL Links              https://www.utdallas.edu/fact-sheets/is/ba-ame...
Department Initials                                                   is
Title                               Bachelor of Arts in American Studies
Catalog Page           https://catalog.utdallas.edu/now/undergraduate...
Name: 0, dtype: object
URL Links              https://www.utdallas.edu/fact-sheets/is/bs-hea...
Department Initials                                                   is
Title                          Bachelor of Science in Healthcare Studies
Catalog Page           https://catalog.utdallas.edu/now/undergraduate...
Name: 1, dtype: object
URL Links              https://www.utdallas.edu/fact-sheets/is/ba-bs-...
Department Initials                                                   is
Title                  Bachelor of Arts and Bachelor of Science in In...
Catalog Page           https://catalog.utdallas.edu/now/undergraduate...
Name: 2, dtype: object
URL Links              https://www.utdallas.edu/fact-sh

TypeError: ignored

In [25]:
# Note, last degree plan (BA-BS In Education) automatically rerouted to the course catalog of it.
link = 'https://catalog.utdallas.edu/2021/undergraduate/programs/is/education/'
# page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')
df['Catalog Page'][140] = link
df['Title'][140] = soup.find('h1').text
df.to_csv('new output.csv',index=False)
print(df.iloc[140])

URL Links              https://www.utdallas.edu/fact-sheets/is/ba-bs-...
Department Initials                                                   is
Title                                School of Interdisciplinary Studies
Catalog Page           https://catalog.utdallas.edu/2021/undergraduat...
Name: 140, dtype: object
